# Metody Obliczeniowe w Nauce i Technice
# Laboratorium 2: Rozwiązywanie układów równań liniowych
## Przemysław Roman

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import time
from copy import deepcopy

def time_exec(func, *args):
    start_time = time.time()
    result = func(*args)
    return result, time.time() - start_time

## Zadanie 1 Metoda Gaussa-Jordana

Napisz i sprawdź funkcję rozwiązującą układ równań liniowych n × n metodą Gaussa-
Jordana. Dla rozmiarów macierzy współczynników większych niż 500 × 500 porównaj
czasy działania zaimplementowanej funkcji z czasami uzyskanymi dla wybranych funkcji
bibliotecznych.

Funkcje pomocnicze

In [19]:
# układa rzędy tak aby ilość wiodących zer w rzędach była niemalejąca idąc od góry do dołu układu równań
def order_rows_by_leading_zeros(AB, n):
  pre_zeros = [0] * n

  for i in range(n):
    for j in range(n):
      if AB[i][j] != 0:
        break
      pre_zeros[i] += 1

  sorted_indexes = [i for _, i in sorted(zip(pre_zeros, range(n)))]
  return np.array([AB[i] for i in sorted_indexes])

def find_max_abs_scalar_i(AB, n, j):
    max_i = -1
    for i in range(j+1, n):
        if AB[i][j] > AB[max_i][j]:
            max_i = i
    return max_i

def partial_pivoting(AB, n, i):
    max_i = find_max_abs_scalar_i(AB, n, i)
    if max_i != -1:
        AB[[i, max_i]] = AB[[max_i, i]]

Implementacja algorytmu Gaussa-Jordana

In [30]:
# i - rząd, j - kolumna
def gauss_jordan(AB, n):
    AB = order_rows_by_leading_zeros(AB, n)

    for i in range(n):
        partial_pivoting(AB, n, i)

        if AB[i][i] != 0:
            # redukcja pivotu do 1
            scalar = 1 / AB[i][i]
            for j in range(n+1):
                AB[i][j] *= scalar

            # redukcja współczynników do 0, które znajdują się w tej samej kolumnie co pivot
            for k in range(n):
                if k != i:
                    scalar = AB[k][i] / AB[i][i]
                    for j in range(n+1):
                        AB[k][j] -= scalar * AB[i][j]

    return AB[:,-1] # ostatnia kolumna zawiera wyniki

Dane testowe

In [31]:
n = 500
A = np.random.random((n, n))
B = np.random.random((n, 1))
AB = np.concatenate((A, B), axis=1)

Testy poprawności rozwiązania oraz czasu wykonania

In [50]:
def get_absolute_errors_sum(vals, expected_vals):
    return np.sum(np.absolute(vals - expected_vals))

X1, t1 = time_exec(gauss_jordan, deepcopy(AB), n)
print(f'Moja implementacja: {t1}[s]')
X2, t2 = time_exec(np.linalg.solve, A, B)
X2 = X2.flatten()
print(f'np.linalg.solve: {t2}[s]')
print('Sumaryczny błąd względny:', get_absolute_errors_sum(X1, X2))

Moja implementacja: 68.61268734931946[s]
np.linalg.solve: 0.002306699752807617[s]
Sumaryczny błąd względny: 2.037349022127044e-11
